In [212]:
from models.PCA import PCA
import pandas as pd
import numpy as np
from tqdm import tqdm
from utils import charas
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [232]:
mon_list = pd.read_pickle('data/mon_list.pkl')

In [233]:
mon_list

0      19570329
1      19570430
2      19570531
3      19570628
4      19570731
         ...   
713    20160831
714    20160930
715    20161031
716    20161130
717    20161230
Name: DATE, Length: 718, dtype: int64

In [234]:
def model_inference_and_predict(model):
    mon_list = pd.read_pickle('data/mon_list.pkl')
    test_mons = mon_list.loc[mon_list >= model.test_period[0]]
    inference_result = []
    predict_result = []
    T_bar = tqdm(test_mons.groupby(test_mons.apply(lambda x: x//10000)), colour='red', desc=f'{model.name} Inferencing & Predicting')
    
    for g in T_bar: # rolling train
        T_bar.set_postfix({'Year': g[0]})
        model.train_model()
        
        for m in g[1].to_list():
            inference_result.append(model.inference(m))
            predict_result.append(model.predict(m))
        # model refit (change train period and valid period)
        model.refit()

    inference_result = pd.DataFrame(inference_result, index=test_mons, columns=charas)
    inference_result.to_csv(f'results/inference/{model.name}_inference.csv')
    
    predict_result = pd.DataFrame(predict_result, index=test_mons, columns=charas)
    predict_result.to_csv(f'results/predict/{model.name}_predict.csv')

In [235]:
pca_1 = PCA(K=1, portfolio=True)
pca_2 = PCA(K=2, portfolio=True)
pca_3 = PCA(K=3, portfolio=True)
pca_4 = PCA(K=4, portfolio=True)
pca_5 = PCA(K=5, portfolio=True)
pca_6 = PCA(K=6, portfolio=True)

In [236]:
model_inference_and_predict(pca_1)
# model_inference_and_predict(pca_2)
# model_inference_and_predict(pca_3)
# model_inference_and_predict(pca_4)
# model_inference_and_predict(pca_5)
# model_inference_and_predict(pca_6)

PCA_1 Inferencing & Predicting:   0%|          | 0/30 [00:00<?, ?it/s, Year=1987]


AttributeError: 'PCA' object has no attribute 'train_model'

In [ ]:
(30 * 12) * N

In [ ]:
def model_inference_and_predict_CA(model):
    mon_list = pd.read_pickle('data/mon_list.pkl')
    test_mons = mon_list.loc[mon_list >= model.test_period[0]]
    inference_result = []
    predict_result = []
    T_bar = tqdm(test_mons.groupby(test_mons.apply(lambda x: x//10000)), colour='red', desc=f'{model.name} Inferencing & Predicting')
    
    for g in T_bar: # rolling train
        T_bar.set_postfix({'Year': g[0]})
        model.train_model()
        
        for m in g[1].to_list():
            stock_index = 
            inference_result.append(model.inference(m))
            predict_result.append(model.predict(m))
        # model refit (change train period and valid period)
        model.refit()

    inference_result = pd.DataFrame(inference_result, index=test_mons, columns=charas)
    inference_result.to_csv(f'results/inference/{model.name}_inference.csv')
    
    predict_result = pd.DataFrame(predict_result, index=test_mons, columns=charas)
    predict_result.to_csv(f'results/predict/{model.name}_predict.csv')

In [ ]:
cal_delayed_fator(month):
    
last_mon = int(str(pd.to_datetime(str(month)) - relativedelta(months=1)).split(' ')[0].replace('-', ''))

_get_itme(last_mon) => fator_nn_input

=> self.factor_nn(fator_nn_input)